# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

X_train.columns = X_train.columns.astype(str)
X_val.columns = X_val.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [3]:
# Your code here

# Scale X_train and X_val using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train = pd.DataFrame(X_train_scaled, columns=X.columns)
X_val = pd.DataFrame(X_val_scaled, columns=X.columns)

In [4]:
# Your code here

# Create a LinearRegression model and fit it on scaled training data
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Calculate a baseline r-squared score on training data
linreg.score(X_train, y_train)


0.7868344817421309

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [5]:
# Your code here

# Set up data structure
interaction_scores = []

# Find combinations of columns and loop over them
combos = list(combinations(X_train.columns, 2))
for (col_1, col_2) in combos:
    # Make copies of X_train and X_val
    X_train_interaction = X_train.copy()
    X_val_interaction = X_val.copy()
    
    # Add interaction term to data
    X_train_interaction['interaction'] = X_train_interaction[col_1] * \
    X_train_interaction[col_2]

    X_val_interaction['interaction'] = X_val_interaction[col_1] * \
    X_val_interaction[col_2]
    
    # Find r-squared score (fit on training data, evaluate on validation data)
    linreg = LinearRegression()
    linreg.fit(X_train_interaction, y_train)
    r_squared = linreg.score(X_val_interaction, y_val)
    
    # Append to data structure
    interaction_scores.append(((col_1, col_2), r_squared))
    
# Sort and subset the data structure to find the top 7
top_7 = sorted(interaction_scores, key=lambda record: record[1], reverse=True)[:7]
top_7

[(('LotArea', '1stFlrSF'), 0.7211105666140574),
 (('LotArea', 'TotalBsmtSF'), 0.7071649207050115),
 (('LotArea', 'GrLivArea'), 0.6690980823779029),
 (('LotArea', 'Fireplaces'), 0.6529699515652589),
 (('2ndFlrSF', 'TotRmsAbvGrd'), 0.6472994890405193),
 (('OverallCond', 'TotalBsmtSF'), 0.642901987923377),
 (('OverallQual', '2ndFlrSF'), 0.6422324294284368)]

### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [6]:
top_7[1][0][1]

'TotalBsmtSF'

In [7]:
# Your code here

# Loop over top 7 interactions
for record in top_7:

    # Extract column names from data structure
    col1 = record[0][0]
    col2 = record[0][1]

    # Construct new column name
    new_col_name = col1 + '_' + col2
    
    # Add new column to X_train and X_val
    X_train[new_col_name] = X_train[col1] * X_train[col2]
    X_val[new_col_name] = X_val[col1] * X_val[col2]

X_train

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820,0.092318,0.073336,-0.057254,0.114116,0.316257,0.325573,-0.125956
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,0.043694,0.175804,0.295392,-0.426859,-0.510770
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820,0.081045,0.003094,0.232730,0.245060,0.793375,-0.016386,0.672141
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023,0.230591,0.128368,0.433899,-0.222636,0.793375,-0.442323,0.672141
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820,0.014341,0.027589,0.005250,0.010842,0.138010,-3.302627,-0.860799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,-0.532331,-0.099842,-0.509252,-0.510628,-0.897228,-0.808132,-1.353116,-2.214167,-0.274466,0.588023,0.477622,0.271823,0.720306,-0.313023,1.789339,0.260039,0.080686
817,-0.309245,-0.099842,-0.509252,0.514106,0.315353,-0.808132,-0.481708,-0.365525,0.088703,-0.994820,-0.097522,-0.158985,0.148966,0.307643,0.295392,-0.261809,0.080686
818,0.119419,0.632038,-0.509252,-0.513011,-0.899947,1.684999,0.796096,0.866903,-0.207566,0.588023,-0.107471,-0.061263,0.095069,0.070221,1.460730,0.261252,1.064983
819,-0.002718,-0.099842,1.304613,-0.889542,-1.329516,0.783758,-0.290233,-0.365525,-0.852668,-0.994820,0.003613,0.002418,0.000789,0.002704,-0.286483,-1.160508,-0.078252


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [8]:
# Your code here

# Set up data structure
polynomials = []

# Loop over all columns
for col in X_train.columns:

    # Loop over degrees 2, 3, 4
    for i in (2, 3, 4):
        
        # Make a copy of X_train and X_val
        features_train = X_train.copy()
        features_val = X_val.copy()
    
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(i, include_bias=False)
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        # poly.fit_transform(features_train[[col]])
        # poly.transform(features_val[[col]])

        # train_transformed = pd.DataFrame(features_train)
        # test_transformed = pd.DataFrame(features_val)

        train_transformed = pd.DataFrame(poly.fit_transform(features_train[[col]]))
        test_transformed = pd.DataFrame(poly.transform(features_val[[col]]))
        

        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
        
        features_train = pd.concat([features_train.drop(col, axis=1), train_transformed], axis=1)

        
        features_val = pd.concat([features_val.drop(col, axis=1), test_transformed], axis=1)

        features_train.columns = features_train.columns.astype(str)
        features_val.columns = features_val.columns.astype(str)
    
        # Find r-squared score on validation
        linreg = LinearRegression()
        linreg.fit(features_train, y_train)
        score = linreg.score(features_val, y_val)
    
        # Append to data structure
        polynomials.append([col, i, score])

# Sort and subset the data structure to find the top 7
top_7_polys = sorted(polynomials, key=lambda record: record[2], reverse=True)[:7]
top_7_polys

[['GrLivArea', 3, 0.8283344365591125],
 ['OverallQual_2ndFlrSF', 3, 0.8222074062501774],
 ['LotArea_Fireplaces', 4, 0.8126598088698527],
 ['LotArea_Fireplaces', 3, 0.8124111133929912],
 ['OverallQual', 3, 0.8068023565969225],
 ['OverallQual_2ndFlrSF', 2, 0.8057607786926371],
 ['OverallQual', 4, 0.803345537886686]]

### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [9]:
# Your code here
top_polys = pd.DataFrame(top_7_polys, columns=('Col', 'Degree', 'R2'))

# Filter out duplicates
top_polys.drop_duplicates(subset='Col', inplace=True)

# Loop over remaining results
for (col, deg, r2) in top_polys.values:
    poly = PolynomialFeatures(deg, include_bias=False)

    # Create polynomial terms
    train_poly = pd.DataFrame(poly.fit_transform(X_train[[col]]), columns=poly.get_feature_names_out([col]))
    val_poly = pd.DataFrame(poly.transform(X_val[[col]]), columns=poly.get_feature_names_out([col]))
    
    # Concat new polynomials to X_train and X_val
    X_train = pd.concat([X_train.drop(col, axis=1), train_poly], axis=1)
    X_val = pd.concat([X_val.drop(col, axis=1), val_poly], axis=1)
    

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [10]:
# Your code here
X_train.head()

,LotArea,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,...,OverallQual_2ndFlrSF,OverallQual_2ndFlrSF^2,OverallQual_2ndFlrSF^3,LotArea_Fireplaces,LotArea_Fireplaces^2,LotArea_Fireplaces^3,LotArea_Fireplaces^4,OverallQual,OverallQual^2,OverallQual^3
0,-0.114710,-0.509252,-0.639316,-0.804789,1.261552,0.250689,0.327629,-0.994820,0.092318,0.073336,...,-0.125956,0.015865,-0.001998,0.114116,0.013022,0.001486,1.695855e-04,-0.099842,0.009968,-0.000995
1,-0.176719,-0.509252,0.838208,0.641608,-0.808132,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,...,-0.510770,0.260886,-0.133253,0.175804,0.030907,0.005434,9.552459e-04,0.632038,0.399472,0.252481
2,-0.246336,1.304613,-0.012560,-0.329000,-0.808132,-0.981739,-1.105931,-0.994820,0.081045,0.003094,...,0.672141,0.451774,0.303656,0.245060,0.060055,0.014717,3.606557e-03,-0.831723,0.691762,-0.575354
3,-0.378617,1.304613,-0.339045,-0.609036,-0.808132,-0.981739,-1.134602,0.588023,0.230591,0.128368,...,0.672141,0.451774,0.303656,-0.222636,0.049567,-0.011035,2.456852e-03,-0.831723,0.691762,-0.575354
4,-0.010898,1.304613,-2.531499,-1.315922,0.550523,0.250689,-2.281450,-0.994820,0.014341,0.027589,...,-0.860799,0.740974,-0.637829,0.010842,0.000118,0.000001,1.381725e-08,-1.563603,2.444854,-3.822780


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [11]:
# Your code here
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print('Train R2: ', linreg.score(X_train, y_train))
print('Test R2: ', linreg.score(X_val, y_val))

Train R2:  0.8571817758242435
Test R2:  0.6442143449163966


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [12]:
# Your code here
for n in (1, 5, 10, 15, 20, 25):
    rfe = RFE(LinearRegression(), n_features_to_select=n)
    X_rfe_train = rfe.fit_transform(X_train, y_train)
    X_rfe_val = rfe.transform(X_val)
    
    lr = LinearRegression()
    lr.fit(X_rfe_train, y_train)
    
    print('R2 Train: ', lr.score(X_rfe_train, y_train))
    print('R2 Val: ', lr.score(X_rfe_val, y_val))
    print(len(X_train.columns) - n, ' features remain')
    print('---------------------')

R2 Train:  0.626356624409971
R2 Val:  0.5883499799376483
25  features remain
---------------------
R2 Train:  0.776039994126505
R2 Val:  0.635298172527236
21  features remain
---------------------
R2 Train:  0.8191862278324272
R2 Val:  0.6743476159860744
16  features remain
---------------------
R2 Train:  0.8483321237427194
R2 Val:  0.7040137671087114
11  features remain
---------------------
R2 Train:  0.8495176468836854
R2 Val:  0.7169477986870848
6  features remain
---------------------
R2 Train:  0.8571732578218183
R2 Val:  0.645929169365514
1  features remain
---------------------


Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [13]:
# Your code here
for a in (1, 10, 50, 100, 200, 500, 1000, 10000):
    lasso = Lasso(alpha=a)
    lasso.fit(X_train, y_train)

    print("Train R^2:", lasso.score(X_train, y_train))
    print("Validation R^2: ", lasso.score(X_val, y_val))
    print(f"Using { 26 - (sum(abs(lasso.coef_) < 10**(-10)))} out of {X_train.shape[1]} features")
    print("and an alpha of", a)
    print()

Train R^2: 0.857153074119144
Validation R^2:  0.648569911635509
Using 26 out of 26 features
and an alpha of 1

Train R^2: 0.8571373079024015
Validation R^2:  0.648052718018303
Using 26 out of 26 features
and an alpha of 10

Train R^2: 0.8570917637035069
Validation R^2:  0.6478656119038081
Using 24 out of 26 features
and an alpha of 50

Train R^2: 0.8569587446238008
Validation R^2:  0.6471042867008596
Using 25 out of 26 features
and an alpha of 100

Train R^2: 0.856449283937646
Validation R^2:  0.6534093776958865
Using 24 out of 26 features
and an alpha of 200

Train R^2: 0.853619971262918
Validation R^2:  0.6778904222044633
Using 22 out of 26 features
and an alpha of 500

Train R^2: 0.8506404012942794
Validation R^2:  0.7222278677869796
Using 17 out of 26 features
and an alpha of 1000

Train R^2: 0.7790529223548714
Validation R^2:  0.7939567393897818
Using 12 out of 26 features
and an alpha of 10000



Compare the results. Which features would you choose to use?

In [14]:
# Your written answer here
"""
I would choose to use the Lasso model with an alpha of 10,000 using 12 features. 
"""

'\nI would choose to use the Lasso model with an alpha of 10,000 using 12 features. \n'

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [15]:
# Your code here

# Scale X_test
# Done at beginning of lab

# Add interactions to X_test
for record in top_7:
    col1, col2 = record[0]
    new_col_name = col1 + "_" + col2
    X_test[new_col_name] = X_test[col1] * X_test[col2]

# Add polynomials to X_test
for (col, degree, r2) in top_polys.values:
    poly = PolynomialFeatures(degree, include_bias=False)
    
    poly_test = pd.DataFrame(poly.fit_transform(X_test[[col]]), columns=poly.get_feature_names_out([col]))
    X_test = pd.concat([X_test.drop(col, axis=1), poly_test], axis=1)


ValueError: Input X contains NaN.
PolynomialFeatures does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [ ]:
# Your code here
final_model = Lasso(alpha=10000)
final_model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

print("R-Squared:", final_model.score(X_test, y_test))
print("MSE:", mean_squared_error(y_test, final_model.predict(X_test)))

## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.